In [1]:
!pip install MiniSom


  Preparing metadata (setup.py) ... done
  Created wheel for MiniSom: filename=MiniSom-2.3.1-py3-none-any.whl size=10589 sha256=8fff0f2ce621e843322f2440ddf06af35924c2ea1eb01004c0953567f3833d68
  Stored in directory: /root/.cache/pip/wheels/c7/92/d2/33bbda5f86fd8830510b16aa98c8dd420129b5cb24248fd6db
Successfully built MiniSom


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from minisom import MiniSom
import numpy as np


In [6]:
!pip install rarfile

import rarfile

# Path to the RAR file
rar_file_path = '/content/drive/MyDrive/Caida 2007/smallest 2 labeled_ddostrace.to-victim.20070804.rar'

# Destination directory for extraction
extracted_directory = '/content/extracted/'

# Create the destination directory if it doesn't exist
import os
os.makedirs(extracted_directory, exist_ok=True)

# Open the RAR file
with rarfile.RarFile(rar_file_path, 'r') as rar:
    # Extract all files to the destination directory
    rar.extractall(extracted_directory)

print(f"Extraction complete. Files are in: {extracted_directory}")


Extraction complete. Files are in: /content/extracted/


In [7]:
import pandas as pd
from minisom import MiniSom
import numpy as np

In [11]:
import numpy as np
import pandas as pd
from minisom import MiniSom
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Load the dataset
# data = pd.read_csv("/content/extracted/labeled_ddostrace.to-victim.20070804.csv")
data = pd.read_csv("/content/labeled_ddostrace.to-victim.20070804 copy.csv")

# Convert other columns to appropriate data types
data['Source IP'] = data['Source IP'].astype(str)
data['Destination IP'] = data['Destination IP'].astype(str)
data['Protocol'] = pd.to_numeric(data['Protocol'], errors='coerce') # if it's numeric
data['Frame Length'] = pd.to_numeric(data['Frame Length'], errors='coerce') # if it's numeric
data['Port Used'] = pd.to_numeric(data['Port Used'], errors='coerce') # if it's numeric
data['Interpacket Time'] = pd.to_numeric(data['Interpacket Time'], errors='coerce') # if it's numeric
data['Entropy'] = pd.to_numeric(data['Entropy'], errors='coerce') # if it's numeric
data['label'] = pd.to_numeric(data['label'], errors='coerce') # if it's numeric

# Drop rows with missing values
data.dropna(inplace=True)






In [12]:
# Separate features and labels
X = data.drop(['label', 'Timestamp', 'Source IP', 'Destination IP'], axis=1).values
y = data['label'].values

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Define SOM parameters
som_grid_rows = 10
som_grid_columns = 10
input_len = X_scaled.shape[1]

# Train SOM
som = MiniSom(som_grid_rows, som_grid_columns, input_len, sigma=1.0, learning_rate=0.5)
som.random_weights_init(X_scaled)

# Define number of iterations and print frequency
num_iterations = 10000
print_frequency = 1000

# Training SOM with progress indicator
for i in range(num_iterations):
    percent_complete = (i + 1) / num_iterations * 100
    if (i + 1) % print_frequency == 0:
        print(f"Training SOM: {percent_complete:.2f}% complete")
    som.train_random(X_scaled, 1)

print("Training SOM: 100.00% complete")

# Find the winning neurons for each sample
winning_neurons = np.array([som.winner(x) for x in X_scaled])

# Assign labels to clusters based on the majority label of the samples in each cluster
cluster_labels = []
total_samples = len(winning_neurons)


Training SOM: 10.00% complete
Training SOM: 20.00% complete
Training SOM: 30.00% complete
Training SOM: 40.00% complete
Training SOM: 50.00% complete
Training SOM: 60.00% complete
Training SOM: 70.00% complete
Training SOM: 80.00% complete
Training SOM: 90.00% complete
Training SOM: 100.00% complete
Training SOM: 100.00% complete


In [13]:
for i, neuron in enumerate(winning_neurons):
    cluster_samples_indices = np.where((winning_neurons[:, 0] == neuron[0]) & (winning_neurons[:, 1] == neuron[1]))[0]
    cluster_labels.append(y[cluster_samples_indices].mean())

    # Print progress
    if (i + 1) % print_frequency == 0 or i + 1 == total_samples:
        percent_complete = (i + 1) / total_samples * 100
        print(f"Assigning cluster labels: {percent_complete:.2f}% complete")



Assigning cluster labels: 66.67% complete
Assigning cluster labels: 100.00% complete


In [14]:
# Convert labels to binary (assuming 0 for non-DDoS and 1 for DDoS)
y_binary = np.where(y > 0, 1, 0)

# Convert cluster labels to binary
cluster_labels_binary = np.where(np.array(cluster_labels) > 0, 1, 0)

# Evaluate accuracy
accuracy = accuracy_score(y_binary, cluster_labels_binary)
print("Accuracy:", accuracy)

Accuracy: 0.9973333333333333


In [ ]:
# Separate features and labels
X = data.drop(['label', 'Timestamp', 'Source IP', 'Destination IP'], axis=1).values
y = data['label'].values

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Define SOM parameters
som_grid_rows = 10
som_grid_columns = 10
input_len = X_scaled.shape[1]
num_iterations = 10000

# Train SOM
som = MiniSom(som_grid_rows, som_grid_columns, input_len, sigma=1.0, learning_rate=0.5)
som.random_weights_init(X_scaled)
som.train_batch(X_scaled, num_iterations)

# Find the winning neurons for each sample
winning_neurons = np.array([som.winner(x) for x in X_scaled])

# Assign labels to clusters based on the majority label of the samples in each cluster
cluster_labels = []
total_samples = len(winning_neurons)

for i, neuron in enumerate(winning_neurons):
    cluster_samples_indices = np.where((winning_neurons[:, 0] == neuron[0]) & (winning_neurons[:, 1] == neuron[1]))[0]
    cluster_labels.append(y[cluster_samples_indices].mean())

    # Print progress
    if (i + 1) % 1000 == 0 or i + 1 == total_samples:
        percent_complete = (i + 1) / total_samples * 100
        print(f"Assigning cluster labels: {percent_complete:.2f}% complete")



In [ ]:
# Evaluate accuracy
accuracy = accuracy_score(y, cluster_labels)
print("Accuracy:", accuracy)

In [ ]:
# Separate features and labels
X = data.drop(['label', 'Timestamp', 'Source IP', 'Destination IP'], axis=1).values
y = data['label'].values

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Define SOM parameters
som_grid_rows = 10
som_grid_columns = 10
input_len = X_scaled.shape[1]

# Train SOM
som = MiniSom(som_grid_rows, som_grid_columns, input_len, sigma=1.0, learning_rate=0.5)
som.random_weights_init(X_scaled)

In [ ]:
# Define number of iterations and print frequency
num_iterations = 100000
print_frequency = 1000



In [ ]:
# Training SOM with progress indicator
for i in range(num_iterations):
    percent_complete = (i + 1) / num_iterations * 100
    if (i + 1) % print_frequency == 0:
        print(f"Training SOM: {percent_complete:.2f}% complete")
    som.train_random(X_scaled, 1)

print("Training SOM: 100.00% complete")

# Find the winning neurons for each sample
winning_neurons = np.array([som.winner(x) for x in X_scaled])

# Assign labels to clusters based on the majority label of the samples in each cluster
cluster_labels = []
total_samples = len(winning_neurons)

for i, neuron in enumerate(winning_neurons):
    cluster_samples_indices = np.where((winning_neurons[:, 0] == neuron[0]) & (winning_neurons[:, 1] == neuron[1]))[0]
    cluster_labels.append(y[cluster_samples_indices].mean())

    # Print progress
    if (i + 1) % print_frequency == 0 or i + 1 == total_samples:
        percent_complete = (i + 1) / total_samples * 100
        print(f"Assigning cluster labels: {percent_complete:.2f}% complete")


Training SOM: 1.00% complete
Training SOM: 2.00% complete
Training SOM: 3.00% complete
Training SOM: 4.00% complete
Training SOM: 5.00% complete
Training SOM: 6.00% complete
Training SOM: 7.00% complete
Training SOM: 8.00% complete
Training SOM: 9.00% complete
Training SOM: 10.00% complete
Training SOM: 11.00% complete
Training SOM: 12.00% complete
Training SOM: 13.00% complete
Training SOM: 14.00% complete
Training SOM: 15.00% complete
Training SOM: 16.00% complete
Training SOM: 17.00% complete
Training SOM: 18.00% complete
Training SOM: 19.00% complete
Training SOM: 20.00% complete
Training SOM: 21.00% complete
Training SOM: 22.00% complete
Training SOM: 23.00% complete
Training SOM: 24.00% complete
Training SOM: 25.00% complete
Training SOM: 26.00% complete
Training SOM: 27.00% complete
Training SOM: 28.00% complete
Training SOM: 29.00% complete
Training SOM: 30.00% complete
Training SOM: 31.00% complete
Training SOM: 32.00% complete
Training SOM: 33.00% complete
Training SOM: 34.00

In [ ]:
# Evaluate accuracy
accuracy = accuracy_score(y, cluster_labels)
print("Accuracy:", accuracy)